In [6]:
import numpy as np

In [135]:
class Card:
    
    def __init__(self,val,suit):
        if val < 1 or val > 13:
            print(f'value must be between 1 and 13 inclusive! You entered val = {val}')
            raise ValueError
        if suit not in 'HDCS':
            print(f'suit must be H (heart), D (diamond), C (club), or S (spade). You entered suit = {suit}')
            raise ValueError
        self.val = val
        self.suit = suit
        self.color = 'Black'
        if self.suit in 'HD':
            self.color = 'Red'
    
    def __str__(self):
        return f'{self.val}{self.suit}'

class Deck:
    
    _rng = np.random.default_rng(1)
    
    def __init__(self):
        self.cards = np.array([Card(val,suit) for val in range(1,14) for suit in 'HDCS'])
        type(self)._rng.shuffle(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])

    def __len__(self):
        return len(self.cards)

    def __getitem__(self,index):
        return self.cards[index]
    
    def draw(self,n):
        if len(self) == 0:
            print('WARNING! Trying to draw from empty deck')
        drawn = self.cards[:n]
        self.cards = self.cards[n:]
        return drawn

class Pile:
    
    def __init__(self,cards):
        self.showing = [cards[0]] if len(cards) > 0 else []
        self.hidden = cards[1:]
    
    def __str__(self):
        return ' '.join(map(str,self.showing)) + ' || ' + ' '.join(map(str,self.hidden))

    def head(self):
        return self.showing[0]

    def tail(self):
        return self.showing[-1]

    def canAdd(self,card):
        raise NotImplemented

    def add(self,card):
        raise NotImplemented


class BoardPile(Pile):

    def __init__(self,cards):
        super().__init__(cards)
    
    def canAdd(self,card):
        return card.val == self.tail().val-1 and card.color != self.tail().color

    def add(self,card):
        self.showing.append(card)

class AcePile(Pile):

    def __init__(self,cards):
        super().__init__(cards)
    
    def __str__(self):
        if len(self.showing) == 0:
            return '_'
        return str(self.tail())
    
    def canAdd(self,card):
        if len(self.showing) == 0:
            return card.val == 1
        return card.val == self.tail().val + 1 and card.suit == self.tail().suit

    # Assert: len(self.showing) should always be 0 or 1
    def add(self,card):
        if len(self.showing) > 0:
            self.hidden.append(self.showing.pop())
        self.showing.append(card)

class Player:
    
    def __init__(self):
        self.hand = Deck()
        self.board = [BoardPile(self.hand.draw(i)) for i in range(1,8)]
        self.allKingsOut = False

    def show(self):
        for i,pile in enumerate(self.board):
            print(f'Pile {i}: {str(pile)}')
        print('---')
        print('Hand:',self.hand)

    def possibleMoves_pileToPile(self):
        moves = []
        for p1 in self.board:
            for p2 in self.board:
                if p1 == p2:
                    continue
                if p2.canAdd(p1.head()):
                    moves.append({'card':p1.head(),'target':p2})
        return moves

    def possibleMoves_handToPile(self):
        moves = []
        for card in self.hand[2::3]:
            for pile in self.board:
                if pile.canAdd(card):
                    moves.append({'card':card,'target':pile})
        return moves

    def possibleMoves_pileToAces(self,acePiles):
        moves = []
        for pile in self.board:
            for acePile in acePiles:
                if acePile.canAdd(pile.tail()):
                    moves.append({'card':pile.tail(),'target':acePile})
        return moves

    def possibleMoves_handToAces(self,acePiles):
        moves = []
        for card in self.hand[::3]:
            for acePile in acePiles:
                if acePile.canAdd(card):
                    moves.append({'card':card,'target':acePile})
        return moves

    def possibleMoves(self,acePiles):
        return {
            'p2p':self.possibleMoves_pileToPile(),
            'h2p':self.possibleMoves_handToPile(),
            'p2a':self.possibleMoves_pileToAces(acePiles),
            'h2a':self.possibleMoves_handToAces(acePiles)
        }   


class Game:
    
    def __init__(self,nPlayers):
        self.nPlayers = nPlayers
        self.players = [Player() for _ in range(self.nPlayers)]
        self.acePiles = [AcePile([]) for _ in range(4*self.nPlayers)]
    
    def show(self):
        print('===== Aces =====')
        for acePile in self.acePiles:
            print(acePile,end=' ')
        print('\n')
        for i,player in enumerate(self.players):
            print(f'===== Player #{i} =====')
            player.show()
            print('\n')

    def play(self):
        # For 1 player only!! Haven't figured out how want to implement multiple ppl yet
        p = self.players[0]
        # while True:
            # pass

In [ ]:
def showMoves(moves):
    print('Possible Moves:')
    for move in moves:
        print('card',move['card'],'target',move['target'])

# g = Game(1)
# print(list(map(id,g.acePiles)))

while True:
    x = input()
    if x != '':
        break
    g = Game(1)
    # g.acePiles[0].add(Card(1,'H'))
    # g.acePiles[1].add(Card(1,'D'))
    # g.acePiles[1].add(Card(2,'D'))
    # g.acePiles[2].add(Card(1,'C'))
    # g.acePiles[2].add(Card(2,'C'))
    # g.acePiles[2].add(Card(3,'C'))
    # g.acePiles[3].add(Card(1,'S'))
    g.show()    
    # showMoves(g.players[0].possibleMoves_pileToPile())
    # showMoves(g.players[0].possibleMoves_handToPile())
    # showMoves(g.players[0].possibleMoves_pileToAces(g.acePiles))
    showMoves(g.players[0].possibleMoves_handToAces(g.acePiles))

    

===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 7C || 
Pile 1: 3S || 9D
Pile 2: 7S || 9H 4H
Pile 3: 2C || 11C 8H 11S
Pile 4: 12S || 13S 13D 1H 10H
Pile 5: 6S || 8C 2H 2D 3H 5H
Pile 6: 13C || 12H 7H 2S 1S 10S 6H
---
Hand: ['12C', '3D', '5S', '13H', '10D', '9C', '8S', '6C', '11H', '9S', '8D', '7D', '6D', '3C', '10C', '4D', '4S', '1D', '5C', '11D', '1C', '4C', '5D', '12D']


Possible Moves:


In [129]:
x = [1,2,3]
x = []
print(x[2::3])

[]


In [23]:
# FROM ChatGPT

# Test the Card class
def test_card():
    # Test valid card creation
    card = Card(5, 'H')
    assert card.val == 5
    assert card.suit == 'H'
    assert card.color == 'Red'
    assert str(card) == '5H'
    
    card = Card(13, 'S')
    assert card.val == 13
    assert card.suit == 'S'
    assert card.color == 'Black'
    assert str(card) == '13S'
    
    # Test invalid card creation
    try:
        Card(0, 'H')
    except ValueError:
        print("ValueError caught as expected for invalid value")
    
    try:
        Card(14, 'H')
    except ValueError:
        print("ValueError caught as expected for invalid value")
    
    try:
        Card(5, 'X')
    except ValueError:
        print("ValueError caught as expected for invalid suit")

# Test the Deck class
def test_deck():
    # Test deck initialization
    deck = Deck()
    assert len(deck) == 52
    assert isinstance(deck.cards, np.ndarray)
    
    # Test deck shuffle
    deck_str_before = str(deck)
    deck = Deck()  # New deck should be shuffled again
    deck_str_after = str(deck)
    assert deck_str_before != deck_str_after  # Check that the decks are shuffled
    
    # Test draw method
    drawn_cards = deck.draw(5)
    assert len(drawn_cards) == 5
    assert len(deck) == 47  # 52 - 5 = 47
    
    # Test draw more cards than available
    drawn_cards = deck.draw(48)
    assert len(drawn_cards) == 47  # Only 47 cards were left
    assert len(deck) == 0
    
    # Test warning when trying to draw from empty deck
    drawn_cards = deck.draw(1)
    assert len(drawn_cards) == 0


test_card()
test_deck()
print("All card & deck tests passed.")

def testPileAdding():
    d = Deck()
    print(d)
    print('---')
    
    p = Pile(d.cards[:5])
    print(p)
    
    while True:
        c = input('Card: ')
        val = c[:-1]
        suit = c[-1]
        if c == 'break':
            break
        try :
            val = int(val)
        except ValueError:
            print(f'enter a valid number! val = {val}')
        print(p.add(Card(val,suit)))
        print(p)
        print('-----')

def testPile():
    p = AcePile([])
    print(p)
    
    # d = Deck()
    # print(d)
    # print('---')
    # p = HandPile(d.cards[:5])
    # print(p)
    
    while True:
        c = input('Card: ')
        if len(c) == 0 or c == 'break':
            break
        val = c[:-1]
        suit = c[-1]
        try :
            val = int(val)
        except ValueError:
            print(f'enter a valid number! val = {val}')
        c = Card(val,suit)
        print(p.canAdd(c))
        if p.canAdd(c):
            p.add(c)
        print(p)
        print('-----')

value must be between 1 and 13 inclusive! You entered val = 0
ValueError caught as expected for invalid value
value must be between 1 and 13 inclusive! You entered val = 14
ValueError caught as expected for invalid value
suit must be H (heart), D (diamond), C (club), or S (spade). You entered suit = X
ValueError caught as expected for invalid suit
WARNING! Trying to draw from empty deck
All tests passed.


In [ ]:
# Idea for definins strategies:
# they take in a list of possible moves, then output the recommendation

# Plan:
# There is a Game class w/ a bunch of players on it
# That is where the .play() function is which loops through turns until game is over and tracks aces, winners, etc

# Each player is a class w/ their deck of cards, hand, & board